# FLOYao demonstration

In [1]:
using Yao, FLOYao

## Simple example

Build a circuit consisting only of FLO gates:

In [2]:
nq = 4
circuit = chain(nq)

push!(circuit, put(nq, 3=>Rz(0.5)))

xxg1 = kron(nq, 1 => X, 2 => X)
rg = rot(xxg1, 0.)
push!(circuit, rg)  

xyg2 = kron(nq, 2 => X, 3 => Z, 4 => Y)
rg = rot(xyg2, 0.)
push!(circuit, rg)  
push!(circuit, put(nq, 3=>Rz(0.5)))
push!(circuit, put(nq, 1=>Z))

# push!(circuit, put(nq, 2 => Ry(9.)))

yxg3 = kron(nq, 2 => Y, 3 => X)
rg = rot(yxg3, 0.)
push!(circuit, rg)

dispatch!(circuit, :random)
circuit

nqubits: 4
chain
├─ put on (3)
│  └─ rot(Z, 0.7152825671383036)
├─ rot(nqubits: 4
kron
├─ 1=>X
└─ 2=>X, 0.19523245434585268)
├─ rot(nqubits: 4
kron
├─ 2=>X
├─ 3=>Z
└─ 4=>Y, 0.7679444408255987)
├─ put on (3)
│  └─ rot(Z, 0.7108748151789774)
├─ put on (1)
│  └─ Z
└─ rot(nqubits: 4
kron
├─ 2=>Y
└─ 3=>X, 0.6134951589259613)

and an observable that is bilinear in the Majorana operators:

In [3]:
hamiltonian = xxg1 + 3*xyg2 + 0.5*yxg3 + kron(nq, 2=>Z) + π * kron(nq, 3=>Z)

nqubits: 4
+
├─ kron
│  ├─ 1=>X
│  └─ 2=>X
├─ [scale: 3] kron
│     ├─ 2=>X
│     ├─ 3=>Z
│     └─ 4=>Y
├─ [scale: 0.5] kron
│     ├─ 2=>Y
│     └─ 3=>X
├─ kron
│  └─ 2=>Z
└─ [scale: π] kron
      └─ 3=>Z


Now, pipe the zero state through the circuit and measure the expectation value of `hamiltonian`.
First in ordinary Yao

In [4]:
array_reg = zero_state(nq)
apply!(array_reg, circuit)
expect(hamiltonian, array_reg)

3.1456824178915626 + 0.0im

and with FLOYao:

In [5]:
majorana_reg = FLOYao.zero_state(nq)
apply!(majorana_reg, circuit)
expect(hamiltonian, majorana_reg)

3.145682417891561

even gradients are supported:

In [6]:
expect'(hamiltonian, array_reg => circuit)

ArrayReg{2, ComplexF64, Array...}
    active qubits: 4/4
    nlevel: 2 => [1.053539129004653, -0.34692503573790257, -0.6268976918132858, 1.0535391290046536, -1.7972361547597475]

In [7]:
expect'(hamiltonian, majorana_reg => circuit)

MajoranaReg{Float64}(4) => [1.0535391290046525, -0.3469250357379024, -0.6268976918132855, 1.0535391290046525, -1.797236154759746]

as well as sampling:

In [8]:
measure(array_reg, nshots=10)

10-element Vector{DitStr{2, 4, Int64}}:
 0000 ₍₂₎
 0000 ₍₂₎
 0000 ₍₂₎
 1010 ₍₂₎
 0000 ₍₂₎
 0110 ₍₂₎
 0000 ₍₂₎
 0000 ₍₂₎
 0110 ₍₂₎
 0000 ₍₂₎

In [9]:
measure(majorana_reg, nshots=10)

10-element Vector{DitStr{2, 4, BigInt}}:
 0000 ₍₂₎
 0000 ₍₂₎
 0000 ₍₂₎
 1010 ₍₂₎
 0000 ₍₂₎
 0000 ₍₂₎
 0000 ₍₂₎
 0000 ₍₂₎
 0000 ₍₂₎
 0000 ₍₂₎

## VQE for the TFIM

The Hamiltonian is given as 
$$
    H = J ∑_i^{L-1} X_i X_{i+1} + h ∑_i^L Z_i = U + T.
$$
and as Ansatz circuits we use the Hamiltonian Variational Ansatz
$$
    U(\vec θ) = ∏_i^p e^{-iθ_{i,U} U} e^{-iθ_{i,T} T} 
$$
with the initial state being the groundstate of the TFIM at $J = 0$, so $|ψ_i⟩ = |0 ⋯ 0⟩$.

First, we define the Hamiltonian

In [10]:
L = 100 # this is far beyond what is possible with a full wavefunction simulation
J = 1.5 
h = -0.5
p = 10  # number of VQE layers
U = map(1:L-1) do i
    J * kron(L, i => X, i+1 => X)
end |> sum

T = map(1:L) do i
    h * kron(L, i => Z)
end |> sum

hamiltonian = T + U
# not really needed, but here to circumvent some doctest  restrictions
summary(hamiltonian)

"Add{2}"

and the ansatz circuit

In [11]:
circuit = chain(L)
for _ in 1:p
    for i in 1:L-1
        push!(circuit, rot(kron(L, i => X, i+1 => X), 0.))
    end
    for i in 1:L
        push!(circuit, put(L, i => Rz(0.)))
    end
end
length(circuit)

1990

as well as the initial state

In [12]:
reg = FLOYao.zero_state(L)
summary(reg)

"MajoranaReg{Float64}"

Now that we defined the hamiltonian, the ansatz circuit and the initial state
we can perform simple gradient descent on the energy expectation value to find
an approximation to the groundstate of $H$:

In [13]:
iterations = 100
gamma = 5e-2

# set the initial parameters
nparams = nparameters(circuit)
dispatch!(circuit, :random) # fix initial parameters for reproducibility

for i in 1:iterations
    _, grad = expect'(hamiltonian, reg => circuit)
    dispatch!(-, circuit, gamma * grad)
    println("Iteration $i, energy = $(round(expect(hamiltonian, reg => circuit), digits=4))")
end

Iteration 1, energy = -1.6665
Iteration 2, energy = -22.8073
Iteration 3, energy = -41.2395
Iteration 4, energy = -57.0927
Iteration 5, energy = -70.7361
Iteration 6, energy = -82.1889
Iteration 7, energy = -91.4075
Iteration 8, energy = -98.5386
Iteration 9, energy = -103.9672
Iteration 10, energy = -108.1376
Iteration 11, energy = -111.3868
Iteration 12, energy = -113.939
Iteration 13, energy = -115.9615
Iteration 14, energy = -117.5918
Iteration 15, energy = -118.9364
Iteration 16, energy = -120.0712
Iteration 17, energy = -121.0481
Iteration 18, energy = -121.9023
Iteration 19, energy = -122.6591
Iteration 20, energy = -123.3366
Iteration 21, energy = -123.9486
Iteration 22, energy = -124.5059
Iteration 23, energy = -125.0164
Iteration 24, energy = -125.4865
Iteration 25, energy = -125.9213
Iteration 26, energy = -126.3247
Iteration 27, energy = -126.7001
Iteration 28, energy = -127.0502
Iteration 29, energy = -127.3776
Iteration 30, energy = -127.6844
Iteration 31, energy = -127.9

and for good measure, let's have a look at the samples:

In [14]:
samples = measure(reg |> circuit, nshots=10)

10-element Vector{DitStr{2, 100, BigInt}}:
 1111111111111111111111111111111111111001101000000000001111101011001011000110000100000011000010000010 ₍₂₎
 0000000000000000000000000000000000000100000111100010000001000000110111100001111100001100000101010100 ₍₂₎
 0000000000000000000000000000000000000000100001110011101100100000011011001101011001001101000111000100 ₍₂₎
 1111111111111111111111111111111111111111011100010010000000100000001010110000010001101100110010000100 ₍₂₎
 0000000000000000000000000000000000000100100001101111001000011000001010000000001001010111100000011000 ₍₂₎
 1111111111111111111111111111111111111100100001010010111000100000000010100101001011101111111000010001 ₍₂₎
 1111111111111111111111111111111111111000101000000110010011100100011101001100010110110000001010100001 ₍₂₎
 0000000000000000000000000000000000000001010100000111100011000101010101000011111000110000001000000100 ₍₂₎
 0000000000000000000000000000000000000111011000011001100010101000110000000011111000110011000011100010 ₍₂₎
 00

## Support for custom gates

Natively, the only FLO gates that come already shipped with `FLOYao.jl` are these
[Supported gates](@ref). But there are many more FLO gates,
one being for example the `FSWAP` gate which swaps to qubits while making sure
to preserve the fermionic commutation relations

In [15]:
@const_gate FSWAP = [1 0 0 0; 0 0 1 0; 0 1 0 0; 0 0 0 -1]
mat(FSWAP)

4×4 Matrix{ComplexF64}:
 1.0+0.0im  0.0+0.0im  0.0+0.0im   0.0+0.0im
 0.0+0.0im  0.0+0.0im  1.0+0.0im   0.0+0.0im
 0.0+0.0im  1.0+0.0im  0.0+0.0im   0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im  -1.0+0.0im

If a gate defines a matrix representation, as we just did for the `FSWAP`gate,
`FLOYao` supports them out of the box by manually checking if they are a FLO
gate and then computing its matrix representation in the Majorana basis. But
this method is fairly slow---though still poly-time and memory---compared to
directly implementing `unsafe_apply!(::MajoranaReg, ::YourBlock)` and
`instruct!(::MajoranaReg, ::YourBlock)` and will warn you accordingly:

In [16]:
nq = 4
fswap = put(nq, (1, 2) => FSWAP)
mreg = FLOYao.zero_state(nq)
mreg |> put(nq, 2 => X)
mreg |> fswap

┌ Warning: Calling manual instruct!(MajoranaReg{Float64}(4), ComplexF64[1.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im 1.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 1.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im -1.0 + 0.0im], (1, 2)).
│ You can greatly speed up your FLO gates by exactly implementing unsafe_apply!()
│ and instruct!() for them. See FLOYao/src/instruct.jl and  FLOYao/src/apply_composite.jl
│ for how to do that.
└ @ FLOYao /home/yc20910/PhD/Work/code/FLOYao/src/instruct.jl:73


MajoranaReg{Float64} with 4 qubits:
8×8 Matrix{Float64}:
 -2.35415e-16  -4.12493e-16  -1.0          …   0.0   0.0   0.0   0.0
  2.46746e-16  -5.5708e-16   -1.26504e-16      0.0   0.0   0.0   0.0
 -1.0          -1.17708e-16   2.55988e-16      0.0   0.0   0.0   0.0
 -1.85286e-16  -1.0           2.44068e-16      0.0   0.0   0.0   0.0
 -0.0          -0.0          -0.0             -1.0  -0.0  -0.0  -0.0
 -0.0          -0.0          -0.0          …  -0.0  -1.0  -0.0  -0.0
 -0.0          -0.0          -0.0             -0.0  -0.0  -1.0  -0.0
 -0.0          -0.0          -0.0             -0.0  -0.0  -0.0  -1.0

Now, before we fix these warnings, let's see how long the current implementation takes:

In [17]:
using BenchmarkTools
using Suppressor # we don't want to get all the warnings when benchmarking
@benchmark @suppress apply!($mreg, $fswap)

BenchmarkTools.Trial: 6228 samples with 1 evaluation.
 Range (min … max):  708.072 μs …   4.084 ms  ┊ GC (min … max): 0.00% … 71.75%
 Time  (median):     750.930 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   797.582 μs ± 237.622 μs  ┊ GC (mean ± σ):  2.20% ±  6.15%

  ▄▆▅██▇▆▆▅▄▄▃▂▁▁                                               ▂
  ██████████████████▇▇▇▇▆▇▇▇▆▆▇▆▆▆▆▇▇▇█▇██▇█▆▇█▇▇▇▆▆▅▅▆▅▆▆▆▅▄▄▄ █
  708 μs        Histogram: log(frequency) by time       1.22 ms <

 Memory estimate: 338.86 KiB, allocs estimate: 496.

To find out what the matrix representation of the `FSWAP` gate in the Majorana
basis is, it is easiest to retrace what is happening inside
`instruct!(::MajoranaReg, ::AbstractMatrix, locs)`. You can use

In [18]:
@which instruct!(mreg, mat(FSWAP), (1,2))

instruct!(reg::MajoranaReg, gate::AbstractMatrix, locs) in FLOYao at /home/yc20910/PhD/Work/code/FLOYao/src/instruct.jl:66

to find the location of the corresponding code. Now let's copy-paste what we found there:

In [19]:
W = FLOYao.qubit2majoranaevolution(Matrix(fswap.content), fswap.locs)

4×4 Matrix{Float64}:
 -2.35415e-16  -4.12493e-16  -1.0           0.0
  2.46746e-16  -5.5708e-16   -1.26504e-16  -1.0
 -1.0          -1.17708e-16   2.55988e-16  -2.38988e-16
 -1.85286e-16  -1.0           2.44068e-16   2.43374e-16

In [20]:
matlocs = 2*(fswap.locs[1]-1)+1:2(fswap.locs[end])

1:4

this matrix gets left-multiplied to the columns `1:4` in the last line of
`FLOYao.majorana_unsafe_apply!(::MajoranaReg, ::PutBlock)`. So we can instead
implement the action of an `FSWAP` gate on a `MajoranaReg` directly as follows:

In [21]:
function YaoBlocks.unsafe_apply!(reg::MajoranaReg, b::PutBlock{2,2,FSWAPGate})
    FLOYao.areconsecutive(b.locs) || throw(NonFLOException("FSWAP must act on consecutive qubits"))
    instruct!(reg, Val(:FSWAP), b.locs)
end

function Yao.instruct!(reg::MajoranaReg, ::Val{:FSWAP}, locs::Tuple)
    i1, i2 = locs
    row1, row2 = reg.state[2i1-1,:], reg.state[2i1,:]
    row3, row4 = reg.state[2i2-1,:], reg.state[2i2,:]
    reg.state[2i1-1,:] .=  .-row3
    reg.state[2i1,:] .=  .-row4
    reg.state[2i2-1,:] .=  .-row1
    reg.state[2i2,:] .=  .-row2
    return reg
end

In [22]:
@benchmark apply!($mreg, $fswap)

BenchmarkTools.Trial: 10000 samples with 685 evaluations.
 Range (min … max):  182.761 ns …   3.817 μs  ┊ GC (min … max): 0.00% … 94.40%
 Time  (median):     194.991 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   227.040 ns ± 256.722 ns  ┊ GC (mean ± σ):  9.72% ±  8.09%

  █▇▆▇▅▅▄▄▄▃▃▂▁                                       ▁         ▂
  ████████████████▇▆▆▅▄▅▆▄▄▄▂▃▂▅▂▂▄▄▃▄▃▄▄▇█▇▇▆▇▇▆▆▇▅▅▇██▆▇▆▅▆▆▄ █
  183 ns        Histogram: log(frequency) by time        411 ns <

 Memory estimate: 512 bytes, allocs estimate: 4.

Which is indeed a significant speed-up!